<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-1fb_zh3q
  Stored in directory: /tmp/pip-ephem-wheel-cache-20jab7l8/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
directory /content/src already exists
Out bin /content/result.out


In [36]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <float.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 10000000
#define N 512

#define e 2.7182818

#define blockZSize 2

#define blockXSize 64
#define blockYSize 64

#define tileXSize 4
#define tileYSize 4

#define extN (N + 2)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  
	float helper[8 * 8];

  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
      int dAI = iBegin - blockZSize + i;
      int dAJ = jBegin - blockZSize + j;
      if(dAI < 0 || dAI >= extN || dAJ < 0 || dAJ >= extN) {
          helper[i * N1 + j] = FLT_MAX;
      }
      else {
          helper[i * N1 + j] = matrix[extN * dAI + dAJ];
      }
		}
	}
  
  int iSplit = iBegin - blockZSize;
  int jSplit = jBegin - blockZSize;
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {
      int offset = (i + blockZSize - k) % 2;
		  if (isBlack) {
			  offset = 1 - offset;
		  }
        
		  for (int j = k + offset; j < N2 - k; j += 2) {
        int b = N1 * i + j;
        if(helper[b + N1] != FLT_MAX && helper[b - N1] != FLT_MAX && helper[b + 1] != FLT_MAX && helper[b - 1] != FLT_MAX) {
            helper[b] = 0.25 * (
                helper[b + N1]
              + helper[b - N1]
              + helper[b + 1]
              + helper[b - 1] 
              - h2 * f(iSplit + i, jSplit + j)
            );
        }
		  }
	  }
	}

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);
    
  for (int i = iBegin; i < iBegin + tileXSize; i++) {
    int offset = (i + blockZSize - 1) % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
        
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      int helperIndex = N1 * (i - iBegin + blockZSize) + j - jBegin + blockZSize;
			matrix[extN * i + j] = 0.25 * (
          helper[helperIndex + N1]
        + helper[helperIndex - N1]
        + helper[helperIndex + 1]
				+ helper[helperIndex - 1] 
        - h2 * f(i, j)
      );
      helper[helperIndex] =	matrix[extN * i + j];
		}
	}
    
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int i = 0; i < row; i++) {
      matrix[0 * row + i] = u_t((i) * h);
      matrix[(row - 1) * row + i] = u_d((i) * h);
  }

  for(int j = 0; j < col; j++) {
      matrix[j * col + 0] = u_l((j) * h);
      matrix[j * col + row - 1] = u_r((j) * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = 1; i < extN - 1; i++) {
		for (int j = 1; j < extN - 1; j++) {
      float fCurrent = func(i, j);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
    ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;

	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();

    if(i%100 == 0) {
        cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
        float diff = calculateDifference(matrixHost);
        cout << " Difference: " << diff << "iter: " << i;
        if(diff < 0.1) break;

    }  
	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

    myfile.close();
	return 0;
}

' i: 509j: 510 Difference: 7.28896iter: 0 i: 478j: 478 Difference: 6.35314iter: 100 i: 466j: 466 Difference: 6.0117iter: 200 i: 457j: 457 Difference: 5.77145iter: 300 i: 449j: 450 Difference: 5.58079iter: 400 i: 444j: 444 Difference: 5.4216iter: 500 i: 438j: 438 Difference: 5.28366iter: 600 i: 433j: 433 Difference: 5.16195iter: 700 i: 428j: 429 Difference: 5.05236iter: 800 i: 424j: 424 Difference: 4.95258iter: 900 i: 420j: 421 Difference: 4.86098iter: 1000 i: 416j: 417 Difference: 4.77624iter: 1100 i: 413j: 413 Difference: 4.69726iter: 1200 i: 409j: 410 Difference: 4.62327iter: 1300 i: 406j: 406 Difference: 4.55361iter: 1400 i: 403j: 403 Difference: 4.48799iter: 1500 i: 400j: 401 Difference: 4.4259iter: 1600 i: 397j: 398 Difference: 4.36674iter: 1700 i: 395j: 395 Difference: 4.31036iter: 1800 i: 392j: 392 Difference: 4.25666iter: 1900 i: 389j: 390 Difference: 4.20516iter: 2000 i: 387j: 387 Difference: 4.15577iter: 2100 i: 384j: 385 Difference: 4.10843iter: 2200 i: 382j: 382 Difference:

In [35]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <float.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 100
#define N 8

#define e 2.7182818

#define blockZSize 3

#define blockXSize 4
#define blockYSize 4

#define tileXSize 2
#define tileYSize 2

#define extN (N + 2)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  
	float helper[6 * 6];

  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
      int dAI = iBegin - blockZSize + i;
      int dAJ = jBegin - blockZSize + j;
      if(dAI < 0 || dAI >= extN || dAJ < 0 || dAJ >= extN) {
          helper[i * N1 + j] = FLT_MAX;
      }
      else {
          helper[i * N1 + j] = matrix[extN * dAI + dAJ];
      }
		}
	}
  
  int iSplit = iBegin - blockZSize;
  int jSplit = jBegin - blockZSize;
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < k + tileXSize; i++) {
      int offset = (i + blockZSize - k) % 2;
		  if (isBlack) {
			  offset = 1 - offset;
		  }
        
		  for (int j = k + offset; j < k + tileXSize; j += 2) {
        int b = N1 * i + j;
        if(helper[b + N1] != FLT_MAX && helper[b - N1] != FLT_MAX && helper[b + 1] != FLT_MAX && helper[b - 1] != FLT_MAX) {
            helper[b] = 0.25 * (
                helper[b + N1]
              + helper[b - N1]
              + helper[b + 1]
              + helper[b - 1] 
              - h2 * f(iSplit + i, jSplit + j)
            );
        }
		  }
	  }
	}

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);
    
  for (int i = iBegin; i < iBegin + tileXSize; i++) {
    int offset = (i + blockZSize - 1) % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
        
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      int helperIndex = N1 * (i - iBegin + blockZSize) + j - jBegin + blockZSize;
			matrix[extN * i + j] = 0.25 * (
          helper[helperIndex + N1]
        + helper[helperIndex - N1]
        + helper[helperIndex + 1]
				+ helper[helperIndex - 1] 
        - h2 * f(i, j)
      );
      helper[helperIndex] =	matrix[extN * i + j];
		}
	}
    
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int i = 0; i < row; i++) {
      matrix[0 * row + i] = u_t((i) * h);
      matrix[(row - 1) * row + i] = u_d((i) * h);
  }

  for(int j = 0; j < col; j++) {
      matrix[j * col + 0] = u_l((j) * h);
      matrix[j * col + row - 1] = u_r((j) * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = 1; i < extN - 1; i++) {
		for (int j = 1; j < extN - 1; j++) {
      float fCurrent = func(i, j);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
    ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;

     for (size_t i = 0; i < extN; i++) {
		  for (size_t j = 0; j < extN; j++) {
        myfile << matrixHost[i * extN + j] << " ";
		  }
      myfile << "\n";
	 }
    
	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
    
      cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
    float diff = calculateDifference(matrixHost);
    cout << " Difference: " << diff << "iter: " << i;
   for (size_t i = 0; i < extN; i++) {
		  for (size_t j = 0; j < extN; j++) {
        myfile << matrixHost[i * extN + j] << " ";
		  }
      myfile << "\n";
	 }
      
      
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();


    cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
    diff = calculateDifference(matrixHost);
    cout << " Difference: " << diff << "iter: " << i;
   for (size_t i = 0; i < extN; i++) {
		  for (size_t j = 0; j < extN; j++) {
        myfile << matrixHost[i * extN + j] << " ";
		  }
      myfile << "\n";
	 }

	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

    myfile.close();
	return 0;
}

' i: 7j: 8 Difference: 5.29449iter: 0 i: 7j: 7 Difference: 4.76696iter: 0 i: 6j: 7 Difference: 3.96109iter: 1 i: 6j: 6 Difference: 3.6989iter: 1 i: 5j: 6 Difference: 3.27243iter: 2 i: 6j: 6 Difference: 3.21118iter: 2 i: 5j: 6 Difference: 2.95019iter: 3 i: 5j: 5 Difference: 2.80653iter: 3 i: 5j: 6 Difference: 2.60348iter: 4 i: 5j: 5 Difference: 2.54014iter: 4 i: 5j: 6 Difference: 2.27869iter: 5 i: 5j: 5 Difference: 2.26633iter: 5 i: 4j: 5 Difference: 2.02543iter: 6 i: 5j: 5 Difference: 2.00718iter: 6 i: 4j: 5 Difference: 1.80678iter: 7 i: 5j: 5 Difference: 1.77115iter: 7 i: 4j: 5 Difference: 1.60492iter: 8 i: 5j: 5 Difference: 1.56023iter: 8 i: 5j: 4 Difference: 1.42209iter: 9 i: 5j: 5 Difference: 1.37355iter: 9 i: 5j: 4 Difference: 1.25827iter: 10 i: 5j: 5 Difference: 1.20909iter: 10 i: 5j: 4 Difference: 1.11237iter: 11 i: 5j: 5 Difference: 1.06451iter: 11 i: 4j: 5 Difference: 0.982898iter: 12 i: 5j: 5 Difference: 0.937487iter: 12 i: 4j: 5 Difference: 0.868236iter: 13 i: 5j: 5 Differen

In [47]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include <float.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 10000000
#define N 128

#define e 2.7182818

#define blockZSize 4

#define blockXSize 32
#define blockYSize 32

#define tileXSize 4
#define tileYSize 4

#define extN (N + 2)
#define Dx 1
#define h ((float) Dx / (N + 1))
#define h2 powf(h, 2.0)

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * blockXSize + threadIdx.x * tileXSize + 1;
	int jBegin = jGl * blockYSize + threadIdx.y * tileYSize + 1;

  int N1 = tileXSize + blockZSize * 2;
  int N2 = tileYSize + blockZSize * 2;
  
	float helper[10 * 10];

  for(int i = 0; i < N1; i++) {
		for(int j = 0; j < N2; j++) {
      int dAI = iBegin - blockZSize + i;
      int dAJ = jBegin - blockZSize + j;
      if(dAI < 0 || dAI >= extN || dAJ < 0 || dAJ >= extN) {
          helper[i * N1 + j] = FLT_MAX;
      }
      else {
          helper[i * N1 + j] = matrix[extN * dAI + dAJ];
      }
		}
	}
  
  int iSplit = iBegin - blockZSize;
  int jSplit = jBegin - blockZSize;
    
  for(int k = 1; k < blockZSize; k++) {
		for (int i = k; i < N1 - k; i++) {
		  for (int j = k; j < N1 - k; j += 2) {
        int b = N1 * i + j;
        if(helper[b + N1] != FLT_MAX && helper[b - N1] != FLT_MAX && helper[b + 1] != FLT_MAX && helper[b - 1] != FLT_MAX) {
            helper[b] = 0.25 * (
                helper[b + N1]
              + helper[b - N1]
              + helper[b + 1]
              + helper[b - 1] 
              - h2 * f(iSplit + i, jSplit + j)
            );
        }
		  }
	  }
	}

  //printf("iGl : %d, %d, %d, %d, %d, %d, %d \n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack, iBegin, jBegin);
    
   for (int i = iBegin; i < iBegin + tileXSize; i++) {
    int offset = (i + blockZSize - 1) % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
        
		for (int j = jBegin + offset; j < jBegin + tileYSize; j += 2) {
      int helperIndex = N1 * (i - iBegin + blockZSize) + j - jBegin + blockZSize;
			matrix[extN * i + j] = 0.25 * (
          helper[helperIndex + N1]
        + helper[helperIndex - N1]
        + helper[helperIndex + 1]
				+ helper[helperIndex - 1] 
        - h2 * f(i, j)
      );
      helper[helperIndex] =	matrix[extN * i + j];
		}
	}
    
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];

	for(int i = 0; i < row; i++) {
		for(int j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}

  for(int i = 0; i < row; i++) {
      matrix[0 * row + i] = u_t((i) * h);
      matrix[(row - 1) * row + i] = u_d((i) * h);
  }

  for(int j = 0; j < col; j++) {
      matrix[j * col + 0] = u_l((j) * h);
      matrix[j * col + row - 1] = u_r((j) * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
  int tolerancei = 0;
  int tolerancej = 0;
  for (int i = 1; i < extN - 1; i++) {
		for (int j = 1; j < extN - 1; j++) {
      float fCurrent = func(i, j);
      if(abs(matrix1[i * extN + j] - fCurrent) > tolerance) {
          tolerance = abs(matrix1[i * extN + j] - fCurrent);
          tolerancei = i;
          tolerancej = j;
      } 
		}
	}
  cout << " i: " << tolerancei << "j: " << tolerancej;
	return tolerance;
}

int main() {
    ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(extN, extN);
	float* matrixDevice;

	int matrixSize = sizeof(float) * extN * extN;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / blockXSize, N / blockYSize);
	dim3 threads(blockXSize / tileXSize, blockYSize / tileYSize);

	for (size_t i = 0; i < ITERATIONS; i++) {
		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);
		cudaPeekAtLastError();
		cudaDeviceSynchronize();

    if(i%100 == 0) {
        cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);
        float diff = calculateDifference(matrixHost);
        cout << " Difference: " << diff << "iter: " << i;
        if(diff < 0.001) break;

    }  
	}

	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
  cout << " Difference: " << calculateDifference(matrixHost);
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}

    myfile.close();
	return 0;
}

' i: 126j: 125 Difference: 6.93513iter: 0 i: 97j: 96 Difference: 4.08172iter: 100 i: 86j: 85 Difference: 3.34085iter: 200 i: 79j: 78 Difference: 2.87558iter: 300 i: 75j: 74 Difference: 2.50288iter: 400 i: 72j: 71 Difference: 2.17304iter: 500 i: 70j: 71 Difference: 1.87967iter: 600 i: 69j: 68 Difference: 1.62199iter: 700 i: 67j: 68 Difference: 1.39758iter: 800 i: 67j: 68 Difference: 1.20348iter: 900 i: 66j: 67 Difference: 1.03613iter: 1000 i: 66j: 67 Difference: 0.891825iter: 1100 i: 65j: 66 Difference: 0.767734iter: 1200 i: 65j: 66 Difference: 0.660909iter: 1300 i: 65j: 66 Difference: 0.568922iter: 1400 i: 65j: 66 Difference: 0.489733iter: 1500 i: 64j: 65 Difference: 0.421581iter: 1600 i: 64j: 65 Difference: 0.362959iter: 1700 i: 64j: 65 Difference: 0.312485iter: 1800 i: 64j: 65 Difference: 0.269028iter: 1900 i: 64j: 65 Difference: 0.231614iter: 2000 i: 64j: 65 Difference: 0.199402iter: 2100 i: 64j: 65 Difference: 0.171669iter: 2200 i: 64j: 65 Difference: 0.147792iter: 2300 i: 64j: 65 

In [34]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 10 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(14), end='')
  i += 1



             1       1.11752       1.24885       1.39561       1.55962       1.74291       1.94773       2.17663       2.43243       2.71828             
       1.11752             0             0             0             0             0             0             0             0       3.03773             
       1.24885             0             0             0             0             0             0             0             0       3.39472             
       1.39561             0             0             0             0             0             0             0             0       3.79367             
       1.55962             0             0             0             0             0             0             0             0        4.2395             
       1.74291             0             0             0             0             0             0             0             0       4.73772             
       1.94773             0             0             0             0    